In [1]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('ratings.txt', delimiter='\t', header=0)
print("1) 데이터 읽기 결과:")
df.head()

1) 데이터 읽기 결과:


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [3]:
print("2) 전체 데이터 길이 (결측치 제거 전):", len(df))
print("각 컬럼별 결측치 개수:\n", df.isnull().sum())
# 결측치가 있는 행 제거
df = df.dropna()
print("결측치 제거 후 데이터셋 길이:", len(df), "\n")

2) 전체 데이터 길이 (결측치 제거 전): 200000
각 컬럼별 결측치 개수:
 id          0
document    8
label       0
dtype: int64
결측치 제거 후 데이터셋 길이: 199992 



In [4]:
okt = Okt()
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [5]:
def text_preprocess(text):
    """
    1. 한글과 공백을 제외한 모든 문자 제거
    2. Okt를 사용하여 토큰화 (어간 추출 포함)
    3. 불용어 제거
    4. 전처리된 토큰들을 공백으로 결합하여 문자열 반환
    """
    # 한글과 공백만 남기기 (숫자, 영문, 특수문자 제거)
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text)
    tokens = okt.morphs(text, stem=True)
    tokens = [word for word in tokens if word not in stopwords]
    return ' '.join(tokens)

In [6]:
sample_text = df['document'].iloc[0]
print("3) 예제 문장 전처리 결과:")
print("원문:", sample_text)
print("전처리:", text_preprocess(sample_text), "\n")

3) 예제 문장 전처리 결과:
원문: 어릴때보고 지금다시봐도 재밌어요ㅋㅋ
전처리: 어리다 때 보고 지금 다시 보다 재밌다 ㅋㅋ 



In [7]:
df['document2'] = df['document'].apply(text_preprocess)
# 전처리된 데이터프레임을 CSV 파일(rating.csv)로 저장
df.to_csv('rating.csv', index=False, encoding='utf-8')
# 저장한 파일을 다시 불러와 확인
df2 = pd.read_csv('rating.csv', delimiter=',')
print("4) 전처리된 데이터프레임 (rating.csv 재불러오기):")
print(df2.head(), "\n")

4) 전처리된 데이터프레임 (rating.csv 재불러오기):
         id                                           document  label  \
0   8112052                                어릴때보고 지금다시봐도 재밌어요ㅋㅋ      1   
1   8132799  디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...      1   
2   4655635               폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.      1   
3   9251303  와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...      1   
4  10067386                        안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.      1   

                                           document2  
0                           어리다 때 보고 지금 다시 보다 재밌다 ㅋㅋ  
1  디자인 을 배우다 학생 외국 디자이너 그 일군 전통 을 통해 발전 문화 산업 부럽다...  
2                    폴리스스토리 시리즈 부터 뉴 까지 버리다 하나 없다 최고  
3        오다 연기 진짜 개 쩔다 지루하다 생각 몰입 보다 그렇다 이렇다 진짜 영화 지  
4                        안개 자욱하다 밤하늘 뜨다 있다 초승달 같다 영화   



In [8]:
X = df['document2']
y = df['label']
# 학습용 80%, 테스트용 20%로 데이터 분할 (random_state로 재현성 확보)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print("5) 데이터셋 분할 완료. 학습용 데이터 수:", X_train_vec.shape[0], ", 테스트용 데이터 수:", X_test_vec.shape[0], "\n")


5) 데이터셋 분할 완료. 학습용 데이터 수: 159993 , 테스트용 데이터 수: 39999 



In [10]:
knn = KNeighborsClassifier(n_neighbors=3)  # k=3 (필요에 따라 조정)
knn.fit(X_train_vec, y_train)
y_pred_knn = knn.predict(X_test_vec)
acc_knn = accuracy_score(y_test, y_pred_knn)
print("6) KNN 모델 정확도: {:.4f}".format(acc_knn))

6) KNN 모델 정확도: 0.7313


In [11]:
lr = LogisticRegression(max_iter=1000)  # max_iter 증가로 수렴 도모
lr.fit(X_train_vec, y_train)
y_pred_lr = lr.predict(X_test_vec)
acc_lr = accuracy_score(y_test, y_pred_lr)
print("   로지스틱 회귀 모델 정확도: {:.4f}\n".format(acc_lr))

   로지스틱 회귀 모델 정확도: 0.8278



In [12]:
new_review = input("7) 새로운 리뷰를 입력하세요: ")
# 전처리 수행
new_review_processed = text_preprocess(new_review)
# 학습 시 사용한 벡터화 객체를 이용하여 벡터화
new_review_vec = vectorizer.transform([new_review_processed])

In [13]:
pred_knn = knn.predict(new_review_vec)[0]
pred_lr = lr.predict(new_review_vec)[0]

In [14]:
def label_to_sentiment(label):
    return "긍정" if label == 1 else "부정"

print("KNN 모델 예측 결과:", label_to_sentiment(pred_knn))
print("로지스틱 회귀 모델 예측 결과:", label_to_sentiment(pred_lr))

KNN 모델 예측 결과: 긍정
로지스틱 회귀 모델 예측 결과: 긍정
